In [1]:
pip install psycopg2-binary pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd

In [3]:
try:
    connection = psycopg2.connect(
        dbname="KALBE",
        user="postgres",
        password="********",
        host="localhost",
        port="5432"
    )
    print("Connected to the database successfully!")

except psycopg2.Error as e:
    print(f"Error: Unable to connect - {e}")


Connected to the database successfully!


In [4]:
query = "SELECT * FROM public.inventory;"
df = pd.read_sql_query(query, connection)
df

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2344,Blackmores Vit C 500Mg,95000,45
2,2343,Obat Decolgen,15000,50
3,2345,Entrasol Gold 370G,90000,120
4,2343,Obat Decolgen,15000,50


In [5]:
query = "SELECT * FROM public.customer_orders;"
df = pd.read_sql_query(query, connection)
df

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,order_no,purchase_amount,order_date,customer_id,salesman_id
0,10001,150,2022-10-05,2005,3002
1,10009,279,2022-09-10,2001,3005
2,10002,65,2022-10-05,2002,3001
3,10004,110,2022-08-17,2009,3003
4,10007,948,2022-09-10,2005,3002
5,10005,2400,2022-07-27,2007,3001


# Show the Item_name with the highest number in Item_total:

In [6]:
query_highest_item_total = "SELECT item_name FROM public.inventory ORDER BY item_total DESC LIMIT 1;"
highest_item = pd.read_sql_query(query_highest_item_total, connection)
print("Item name with the highest number in Item_total:")
print(highest_item)

Item name with the highest number in Item_total:
            item_name
0  Entrasol Gold 370G


D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Update the Item_price of the output of the previous question:

In [7]:
new_item_price = 100000
query_update_item_price = f"UPDATE public.inventory SET item_price = {new_item_price} WHERE item_name = '{highest_item['item_name'][0]}';"
with connection.cursor() as cursor:
    cursor.execute(query_update_item_price)
connection.commit()
print("Item_price updated successfully.")


Item_price updated successfully.


In [8]:
query = "SELECT * FROM public.inventory;"
df = pd.read_sql_query(query, connection)
df

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2344,Blackmores Vit C 500Mg,95000,45
2,2343,Obat Decolgen,15000,50
3,2343,Obat Decolgen,15000,50
4,2345,Entrasol Gold 370G,100000,120


# Insert a new Item_name with Item_code of 2343 into the table

In [9]:
new_item_values = ("Obat Decolgen", 2343, 50, 15000)
query_insert_new_item = "INSERT INTO public.inventory (item_name, item_code, item_total, item_price) VALUES (%s, %s, %s, %s);"
with connection.cursor() as cursor:
    cursor.execute(query_insert_new_item, new_item_values)
connection.commit()
print("New item inserted successfully.")


New item inserted successfully.


In [10]:
query = "SELECT * FROM public.inventory;"
df = pd.read_sql_query(query, connection)
df

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2344,Blackmores Vit C 500Mg,95000,45
2,2343,Obat Decolgen,15000,50
3,2343,Obat Decolgen,15000,50
4,2345,Entrasol Gold 370G,100000,120
5,2343,Obat Decolgen,15000,50


# Delete the Item_name that has the lowest number of Item_total

In [11]:
try:
    query_delete_lowest_item_total = "DELETE FROM public.inventory WHERE item_total = (SELECT MIN(item_total) FROM public.inventory);"

    with connection.cursor() as cursor:
        cursor.execute(query_delete_lowest_item_total)

    connection.commit()
    print("Item with the lowest Item_total deleted successfully.")

except psycopg2.Error as e:
    connection.rollback()  # Rollback the transaction to clear the error state
    print(f"Error: {e}")

Item with the lowest Item_total deleted successfully.


In [12]:
query = "SELECT * FROM public.inventory;"
df = pd.read_sql_query(query, connection)
df

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,item_code,item_name,item_price,item_total
0,2341,Promag Tablet,3000,100
1,2343,Obat Decolgen,15000,50
2,2343,Obat Decolgen,15000,50
3,2345,Entrasol Gold 370G,100000,120
4,2343,Obat Decolgen,15000,50


# Display all customer orders where
purchase amount < 100 or exclude those
orders which order date is on or greater than 25 Aug
2022 and customer id is above 2001

In [13]:
query = """
SELECT *
FROM customer_orders
WHERE (purchase_amount < 100 OR (order_date < '2022-08-25' AND customer_id <= 2001));
"""

In [15]:
df1 = pd.read_sql_query(query, connection)
df1

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,order_no,purchase_amount,order_date,customer_id,salesman_id
0,10002,65,2022-10-05,2002,3001
